In [1]:
import numpy as np
import pandas as pd

import re

In [2]:
data_path = "/content/drive/MyDrive/Recommender System/Week 2/netflix_movie_recommender/"
file_name = "netflix_titles.csv"

In [8]:
netflix_df = pd.read_csv(data_path + file_name)
netflix_df.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."


In [9]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


In [10]:
netflix_df.dropna(subset=['cast','title','description','listed_in'],inplace=True,axis=0)
data = netflix_df.reset_index(drop=True)

In [11]:
data['listed_in'] = [re.sub(r'[^\w\s]', '', t) for t in data['listed_in']]
data['cast'] = [re.sub(',',' ',re.sub(' ','',t)) for t in data['cast']]
data['description'] = [re.sub(r'[^\w\s]', '', t) for t in data['description']]
data['title'] = [re.sub(r'[^\w\s]', '', t) for t in data['title']]

In [12]:
data["combined"] = data['listed_in'] + '  ' + data['cast'] + ' ' + data['title'] + ' ' + data['description']
data.drop(['listed_in','cast','description'],axis=1,inplace=True)
data.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,combined
0,81145628,Movie,Norm of the North King Sized Adventure,"Richard Finn, Tim Maltby","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,Children Family Movies Comedies AlanMarriott...
1,80117401,Movie,Jandino Whatever it Takes,NaN,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,StandUp Comedy JandinoAsporaat Jandino Whatev...
2,70234439,TV Show,Transformers Prime,NaN,United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids TV PeterCullen SumaleeMontano FrankWelke...
3,80058654,TV Show,Transformers Robots in Disguise,NaN,United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids TV WillFriedle DarrenCriss ConstanceZimm...
4,80125979,Movie,realityhigh,Fernando Lebrija,United States,"September 8, 2017",2017,TV-14,99 min,Comedies NestaCooper KateWalsh JohnMichaelHig...


### Sentence Bert Encoding

In [13]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
movies_encoding = model.encode(data["combined"])

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5664 entries, 0 to 5663
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       5664 non-null   int64 
 1   type          5664 non-null   object
 2   title         5664 non-null   object
 3   director      3909 non-null   object
 4   country       5271 non-null   object
 5   date_added    5654 non-null   object
 6   release_year  5664 non-null   int64 
 7   rating        5657 non-null   object
 8   duration      5664 non-null   object
 9   combined      5664 non-null   object
dtypes: int64(2), object(8)
memory usage: 442.6+ KB


In [34]:
type(movies_encoding), movies_encoding.shape

(numpy.ndarray, (5664, 384))

In [35]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities_sbert = linear_kernel(movies_encoding, movies_encoding)


In [36]:
cosine_similarities_sbert.shape, type(cosine_similarities_sbert)

((5664, 5664), numpy.ndarray)

In [37]:
indices = pd.Series(data.index, index = data["title"])

In [38]:
indices[:2]

title
Norm of the North King Sized Adventure    0
Jandino Whatever it Takes                 1
dtype: int64

In [39]:
movie_title = data["title"]

In [40]:
def content_recommender(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_similarities_sbert[idx]))
  sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
  sim_scores = sim_scores[1:31]
  movie_indices = [i[0] for i in sim_scores]
  return movie_title.iloc[movie_indices]

In [41]:
content_recommender('Friends')

2447                                       5050
984                        Merry Happy Whatever
1952                               5 Flights Up
5223                                   Atypical
1243                             Getting Played
5111                            Toast of London
3173                        Dinner for Schmucks
399                                Private Life
3669                      The Edge of Seventeen
4938                                   Band Aid
2568               The Incredible Jessica James
266                                Just Friends
4346    Wet Hot American Summer Ten Years Later
1069                      Shes Out of My League
2673                                The BreakUp
48                                 Carrie Pilby
1070                                  Superstar
4561                               The Outcasts
2735                               Smart People
4606                               Rumor Has It
3813                              Heartb

**Friends: 5050, merry happy whatever, 5 flights up, atypical are all comedy/drama**

### Tfidf encoding

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [43]:
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(data["combined"])
cosine_similarities_tfidf = linear_kernel(matrix,matrix)

In [44]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities_tfidf[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return movie_title.iloc[movie_indices]

In [45]:
content_recommender("Friends")

5437                                           Episodes
5292                                          Dads Army
2356                                         Life Story
3979                               Single Ladies Senior
19                                    Manhattan Romance
1391                                         Warehoused
5174                                      Little Things
5305                                  Trailer Park Boys
4854                                           Lunatics
2006                                        Satrangi Re
5204            Mystery Science Theater 3000 The Return
4969            Midnight Misadventures With Mallika Dua
702                                     Charlies Angels
5475                                   The Inbetweeners
703                       Charlies Angels Full Throttle
4936                    ONegative Love Cant Be Designed
280                                            Superbad
5287                                            